In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
from dateutil import parser
import pandas as pd
import time, re
import csv
import tweepy as tw
from csv import writer
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Enter Twitter API Keys
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

def urlRemover(txt):
    import re
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [ ]:
def scrapTweet(query):
    driver = webdriver.Chrome('chromedriver')
    driver.base_url = query
    time.sleep(3)
    driver.get(driver.base_url)
    last_height=0
    height="window.scrollTo("+str(last_height)+","+str(last_height+2000)+");"
    tweet_ID_list=[]
    
    while(True):
        driver.execute_script(height)
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height=new_height
        height="window.scrollTo("+str(new_height)+","+str(new_height+2000)+");"
        #print(last_height)
        html_source = driver.page_source
        sourcedata= html_source.encode('utf-8')
        soup=bs(sourcedata)
        tweets=soup.body.findAll(["a"])
        for i in tweets:
            if '/status/' in str(i):
                clean_text= re.search('"auto" href=(.*) id=', str(i))
                if clean_text:
                    clean_text=clean_text.group(1).split('/')
                    tweet_ID_list.append(clean_text[-1][:-1])
                    print('row:',clean_text[-1][:-1])
    driver.close()
    return tweet_ID_list

In [ ]:
def exactword(bigram):
    unibi=bigram.split(' ')
    result='https://twitter.com/search?q="'+unibi[0]+"%20"+unibi[1]+'"&src=typed_query&f=top'
    return result

def exactwordANyword(exactBigram,anywordList):
    #"construction zone" (wisconsin, OR WI, OR chicago, OR milwaukee)
    unibi=exactBigram.split(' ')
    anylist='%20('
    counter=0
    length_anywordList=len(anywordList)
    for i in (anywordList):       
            
        if counter<length_anywordList-1:
            anylist=anylist+i+"%2C%20OR%20"
           
        else:
            anylist=anylist+i+")"            
        counter+=1
    
    result='https://twitter.com/search?q="'+unibi[0]+"%20"+unibi[1]+'"'+anylist+'&src=typed_query&f=top'
    return result

In [ ]:
df=pd.read_csv('bigram_search_for_tweet _usstate.csv',usecols=['Bigrams'])# provide bigram keyword's csv file
dfCity=pd.read_csv('usstate.csv',usecols=['state','name'])
df.Bigrams.values[13:20]

In [ ]:
tweet_ID_list_all=[]
for i in df.Bigrams.values[20:]:    
    for j,k in zip(dfCity.state.values,dfCity.name.values):        
        query=exactwordANyword(i,[j,k])        
        scrapTweet(query)
        tweet_ID_list_all=tweet_ID_list_all+scrapTweet(query)
    
    

In [ ]:
pd.DataFrame({'tweetID':[("id"+i) for i in tweet_ID_list_all]}).to_csv('tweetID_usstate.csv',index=None,mode='a')

In [ ]:
def tweetfromAPi(idlist): 
    c=0
    lengthobsub=100
    while lengthobsub>99:
        #lengthobsub=len(tweet_ID_list_all[c:c+100])
        tweetList=idlist[c:c+100]
        #print(tweetList)
        lengthobsub=len(tweetList)
        c+=100
        try:
            tweets = api.statuses_lookup(tweetList,tweet_mode="extended")
            #print(tweets)
            for tweet in tweets:
                #print(tweet.created_at.date())
                if tweet.place:
                    place=["id"+tweet.id_str,tweet.created_at.date(),tweet.place.full_name+","+tweet.place.country,tweet.full_text]       
                    pd.DataFrame(data=[place ]).to_csv('WZ.csv',mode='a',header=None,index=None)
                else:
                    place=["id"+tweet.id_str,tweet.created_at.date(),"None",tweet.full_text]
                    #print(place)
                    pd.DataFrame(data=[place ]).to_csv('WZ.csv',mode='a',header=None,index=None)

        except:      

            pass


In [ ]:
## it may return duplicate tweets, which need further processing to revome the duplicate tweets.